In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn
from torchvision import datasets, transforms

In [ ]:
mnist_transform=transforms.Compose([
                              transforms.Resize((28,28)),
                              transforms.ToTensor(), # first, convert image to PyTorch (float) tensor
                              transforms.Normalize((0.5,), (0.5,)) # normalize inputs, x' = (x-u)/std, x'=[-1, 1] 
                              ])     

train_dataset = datasets.MNIST(root='./mnist_data', 
                                  train=True, 
                                  download=True, 
                                  transform=mnist_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True) 
# shuffle training data before each epoch to avoid local min

# validation or test 
validation_dataset = datasets.MNIST(root='./mnist_data', 
                                  train=False, 
                                  download=True, 
                                  transform=mnist_transform)

validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=100, shuffle=False) 
#print(train_dataset)
#print(validation_dataset)

# contruct model architecture

In [ ]:
class NN_MNIST(nn.Module):
  def __init__(self, NN_in, H1,H2,H3,NN_out):
    super().__init__()
    self.linear1 = nn.Linear(NN_in, H1)
    self.linear2 = nn.Linear(H1, H2)
    self.linear3 = nn.Linear(H2, H3)
    self.linear4 = nn.Linear(H3, NN_out)

    nn.init.xavier_uniform_(self.linear1.weight)
    nn.init.xavier_uniform_(self.linear2.weight)
    nn.init.xavier_uniform_(self.linear3.weight)
    nn.init.xavier_uniform_(self.linear4.weight)

  def forward(self, x):
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = self.linear4(x) # activation not needed for multi-class classification task
    return x

# make model instance 


In [ ]:
torch.manual_seed(1)
model = NN_MNIST(784,512,256,128,10)
model

NN_MNIST(
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=10, bias=True)
)

# optimizer 

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
criterion = nn.CrossEntropyLoss() # cf. BCE()

In [ ]:
epochs = 40
running_loss_history = []
running_corrects_history = []
validation_running_loss_history = []
validation_running_corrects_history = []

for eno in range(epochs):

  running_loss = 0.0
  running_corrects = 0.0
  validation_running_loss = 0.0
  validation_running_corrects = 0.0

  # a batch of 100 images x 600 batches -> 60,000 for a single epoch 
  for inputs, labels in train_loader:     
    # input [100(batch size), 1(channel), 28(width), 28(height)] =>     
    inputs = inputs.view(inputs.shape[0], -1) # [100, 784]
    outputs = model(inputs) # logits (scores)
    loss = criterion(outputs, labels) # outputs [100, 1], labels [100]
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #max function output(max, max_indices)
    _, pred_idx = torch.max(outputs, 1) # 2nd argument - the dim to reduce    

    running_loss += loss.item() # loss per batch 
    running_corrects += torch.sum(pred_idx == labels.data)
  else:
    with torch.no_grad():
      for validation_inputs, validation_labels in validation_loader:     
        # input [100(batch size), 1(channel), 28(width), 28(height)] =>     
        validation_inputs = validation_inputs.view(validation_inputs.shape[0], -1) # [100, 784]
        validation_outputs = model(validation_inputs) # logits (scores)
        validation_loss = criterion(validation_outputs, validation_labels) # outputs [100, 1], labels [100]
        
        #max function output(max, max_indices)
        _, pred_idx = torch.max(validation_outputs, 1) # 2nd argument - the dim to reduce    

        validation_running_loss += validation_loss.item() # loss per batch 
        validation_running_corrects += torch.sum(pred_idx == validation_labels.data)

    epoch_loss = running_loss/len(train_loader) 
    running_loss_history.append(epoch_loss)
    epoch_acc = (running_corrects.float()/inputs.shape[0]*100)/len(train_loader) 
    running_corrects_history.append(epoch_acc)
    print('--------------- epoch: ', (eno+1), '---------------')
    print('training loss: {:.4f}, accuracy {:.4f}'.format(epoch_loss, epoch_acc.item()))   

    validation_epoch_loss = validation_running_loss/len(validation_loader) 
    validation_running_loss_history.append(validation_epoch_loss)
    validation_epoch_acc = (validation_running_corrects.float()/validation_inputs.shape[0]*100)/len(validation_loader) 
    validation_running_corrects_history.append(validation_epoch_acc)    
    print('validation loss: {:.4f}, accuracy {:.4f}'.format(validation_epoch_loss, validation_epoch_acc.item())) 

--------------- epoch:  1 ---------------
training loss: 0.4116, accuracy 87.1667
validation loss: 0.2225, accuracy 93.2400
--------------- epoch:  2 ---------------
training loss: 0.1713, accuracy 94.8233
validation loss: 0.1256, accuracy 96.2800
--------------- epoch:  3 ---------------
training loss: 0.1234, accuracy 96.1750
validation loss: 0.1132, accuracy 96.5900
--------------- epoch:  4 ---------------
training loss: 0.0959, accuracy 97.0117
validation loss: 0.1134, accuracy 96.4900
--------------- epoch:  5 ---------------
training loss: 0.0801, accuracy 97.6050
validation loss: 0.1031, accuracy 96.5200
--------------- epoch:  6 ---------------
training loss: 0.0665, accuracy 97.9433
validation loss: 0.0830, accuracy 97.4000
--------------- epoch:  7 ---------------
training loss: 0.0571, accuracy 98.2933
validation loss: 0.0853, accuracy 97.2300
--------------- epoch:  8 ---------------
training loss: 0.0485, accuracy 98.5117
validation loss: 0.0846, accuracy 97.3900
--------

# minist data visualization 

In [ ]:
# iter() returns an iterator for the given iterable object
dataiter = iter(validation_loader)
inputs, labels = dataiter.next()
outputs = model(inputs.view(inputs.shape[0], -1)) # logits (scores)
_, pred_idx = torch.max(outputs, 1) # 2nd argument - the dim to reduce    

fig = plt.figure(figsize=(25, 25))

for idx in np.arange(len(dataiter)): 
  ax = fig.add_subplot(10, 10, idx+1, xticks=[], yticks=[])

  image = inputs[idx].clone().detach().numpy() # [1, 28, 28] = [channel, width, height]
  image = image.transpose(1, 2, 0) # [28, 28, 1]
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5)) # denomalization : x' = (x-u)/std => x = x'*std + u
  image = image.clip(0, 1)
  plt.imshow(image)

  ax.set_title("P{} / L{}".format(str(pred_idx[idx].item()), str(labels[idx].item())), color=("red" if pred_idx[idx]!=labels[idx] else "black"))



  
